# RNNs using Pytorch

In [1]:
import pandas as pd

df = pd.read_csv('/content/100_Unique_QA_Dataset.csv')
df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [2]:
# tokenize
def tokenize(text):
  text = text.lower()
  text = text.replace('?','')
  text = text.replace("'","")
  return text.split()

In [3]:
tokenize(df['question'][0])

['what', 'is', 'the', 'capital', 'of', 'france']

In [4]:
# vocab
vocab = {'<UNK>':0}

def build_vocab(row):
  tokenized_question = tokenize(row['question'])
  tokenized_answer = tokenize(row['answer'])
  merged_token = tokenized_question + tokenized_answer
  for token in merged_token:
    if token not in vocab:
      vocab[token] = len(vocab)

In [5]:
df.apply(build_vocab,axis=1)

,0
0,None
1,None
2,None
3,None
4,None
...,...
85,None
86,None
87,None
88,None


In [6]:
len(vocab)

324

In [7]:
# convert
def text_to_indices(text, vocab):
  tokenized_text = tokenize(text)
  indices = []
  for token in tokenized_text:
    if token in vocab:
      indices.append(vocab[token])
    else:
      indices.append(vocab['<UNK>'])
  return indices

In [8]:
text_to_indices('who is sajid', vocab)

[10, 2, 0]

In [9]:
from pandas.core.arrays import numeric
import torch
from torch.utils.data import Dataset, DataLoader

class QADataset(Dataset):

  def __init__(self, df, vocab):
    super().__init__()
    self.df = df
    self.vocab = vocab

  def __len__(self):
    return self.df.shape[0]

  def __getitem__(self, index):
    numericalized_question = text_to_indices(self.df['question'][index], self.vocab)
    numericalized_answer = text_to_indices(self.df['answer'][index], self.vocab)
    return torch.tensor(numericalized_question), torch.tensor(numericalized_answer)


In [10]:
dataset = QADataset(df, vocab)
dataset[10]

(tensor([ 1,  2,  3,  4,  5, 53]), tensor([54]))

In [11]:
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [12]:
for question, answer in dataloader:
  print(question)
  print(answer)

tensor([[  1,   2,   3,   4,   5, 236, 237]])
tensor([[238]])
tensor([[ 1,  2,  3,  4,  5, 73]])
tensor([[74]])
tensor([[ 10, 140,   3, 141, 270,  93, 271,   5,   3, 272]])
tensor([[273]])
tensor([[  1,   2,   3,   4,   5, 286]])
tensor([[287]])
tensor([[ 10, 140,   3, 141, 142,  12, 143,  83,   3, 144]])
tensor([[145]])
tensor([[  1,   2,   3,  69,   5, 155]])
tensor([[156]])
tensor([[ 1,  2,  3,  4,  5, 53]])
tensor([[54]])
tensor([[42, 86, 87, 88, 89, 39, 90]])
tensor([[91]])
tensor([[ 10,  11, 157, 158, 159]])
tensor([[160]])
tensor([[  1,   2,   3, 221,   5, 222, 223, 224]])
tensor([[225]])
tensor([[ 42, 255,   2, 256,  83, 257, 258]])
tensor([[259]])
tensor([[ 10,  11, 189, 158, 190]])
tensor([[191]])
tensor([[10, 29,  3, 30, 31]])
tensor([[32]])
tensor([[ 42,   2,   3, 274, 211, 275]])
tensor([[276]])
tensor([[  1,   2,   3, 122, 123,  19,   3,  45]])
tensor([[124]])
tensor([[ 10,  75, 111]])
tensor([[112]])
tensor([[  1,   2,   3, 141, 117,  83,   3, 277, 278]])
tensor([[121]])

In [13]:
import torch.nn as nn

class SimpleRNN(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim=50)
    self.rnn = nn.RNN(input_size=50, hidden_size=64, batch_first=True)
    self.fc = nn.Linear(64, vocab_size)

  def forward(self, question):
    embedded_question = self.embedding(question)
    hidden, final = self.rnn(embedded_question)
    output = self.fc(final)
    return output


In [14]:
dataset[0][0]

tensor([1, 2, 3, 4, 5, 6])

In [15]:
x = nn.Embedding(324, 50)
x(dataset[0][0]).shape

torch.Size([6, 50])

In [16]:
a = x(dataset[0][0])
y = nn.RNN(50,64)

In [17]:
y(a)[1]

tensor([[ 0.2770, -0.2757,  0.1216, -0.4462,  0.2212, -0.1473, -0.0020, -0.2261,
          0.2824, -0.2916,  0.6869, -0.3327, -0.5919, -0.5621,  0.5231,  0.4842,
         -0.3322,  0.7472, -0.6223,  0.0024,  0.5334,  0.8149, -0.1232,  0.3559,
          0.0251,  0.0626,  0.5804,  0.0208, -0.0277,  0.6957,  0.7736,  0.0389,
         -0.6737,  0.3355, -0.1666, -0.6087,  0.8635, -0.0233, -0.5102,  0.8854,
          0.3881, -0.4307,  0.2883, -0.8430,  0.3817, -0.3329, -0.7809, -0.4635,
         -0.0709, -0.4681,  0.2399,  0.8013, -0.5251, -0.5936,  0.6815, -0.2255,
         -0.2029, -0.2947,  0.1194, -0.3866,  0.1582,  0.1664, -0.6441,  0.1435]],
       grad_fn=<SqueezeBackward1>)

In [18]:
b = y(a)[1]
z = nn.Linear(64,324)

In [19]:
z(b).shape

torch.Size([1, 324])

In [20]:
learning_rate = 0.001
epochs = 20
model = SimpleRNN(len(vocab))
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [21]:
for epoch in range(epochs):

  total_epoch_loss = 0

  for question, answer in dataloader:

    optimizer.zero_grad()
    # forward
    output = model(question)
    # loss
    loss = criterion(output.squeeze(0), answer.squeeze(0))
    # gradients
    loss.backward()
    # update
    optimizer.step()

    total_epoch_loss = total_epoch_loss + loss.item()

  print(f'Epoch: {epoch+1}, Loss: {total_epoch_loss}')


Epoch: 1, Loss: 524.843635559082
Epoch: 2, Loss: 459.0395212173462
Epoch: 3, Loss: 382.57072019577026
Epoch: 4, Loss: 318.228013753891
Epoch: 5, Loss: 266.3976582288742
Epoch: 6, Loss: 217.35566318035126
Epoch: 7, Loss: 173.29490339756012
Epoch: 8, Loss: 134.0852854847908
Epoch: 9, Loss: 102.59512215852737
Epoch: 10, Loss: 78.47663950920105
Epoch: 11, Loss: 59.959540486335754
Epoch: 12, Loss: 46.635952815413475
Epoch: 13, Loss: 36.524316638708115
Epoch: 14, Loss: 29.326014444231987
Epoch: 15, Loss: 23.999031722545624
Epoch: 16, Loss: 19.85263930261135
Epoch: 17, Loss: 16.614073872566223
Epoch: 18, Loss: 14.124020516872406
Epoch: 19, Loss: 12.073077391833067
Epoch: 20, Loss: 10.503679729998112


In [22]:
def prediction(model, question, thresold=0.5):
  # numerical question
  numericalized_question = text_to_indices(question, vocab)
  # tensor question
  question_tensor = torch.tensor(numericalized_question).unsqueeze(0)
  # send to model
  output = model(question_tensor)
  output = output.squeeze(0)
  # convert logits to probability
  probs = nn.functional.softmax(output, dim=1)
  # find index of max  prob
  value, index = torch.max(probs, dim=1)

  if value < thresold:
    print("I don't know")
  else:
    print(list(vocab.keys())[index])

In [23]:
prediction(model, 'What is the boiling point of water in Celsius?')

100
